In [ ]:
# conda install -c conda-forge bayesian-optimization
# conda install -c conda-forge xgboost

In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
# from xgboost import XGBClassifier

In [2]:
lgb.__version__

'2.3.1'

In [3]:
# Reading the saved dtypes Series
final_df_dtypes = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_fromBDSE12_03G_HomeCredit_V2_le_20200210a_dtypes_series.csv'\
            , header=None, index_col=0, squeeze=True)
del final_df_dtypes.index.name
final_df_dtypes = final_df_dtypes.to_dict()

final_df = \
pd.read_csv('../../../BDSE12-Group3/datasets/homecdt_ss_output/ss_fteng_fromBDSE12_03G_HomeCredit_V2_le_20200210a.csv'\
           , dtype= final_df_dtypes)

In [4]:
final_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in final_df.columns]
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 1325 entries, SK_ID_CURR to GOODS_PRICE_PREV__na
dtypes: float64(543), int64(254), uint8(528)
memory usage: 2.3 GB


In [5]:
df = final_df
del final_df
gc.collect()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 1325 entries, SK_ID_CURR to GOODS_PRICE_PREV__na
dtypes: float64(543), int64(254), uint8(528)
memory usage: 2.3 GB


In [6]:
df['TARGET'].value_counts()

0.0    282686
1.0     24825
Name: TARGET, dtype: int64

In [7]:
# scale_pos_weight 
282686 / 24825

11.387150050352467

---

## LGBM

In [8]:
gc.collect()

66

In [10]:
def lgbm_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['max_bin'] = int(params['max_bin'])
    
        
    clf = LGBMClassifier(**params, 
#                          n_estimators = 2000,
#                          nthread = 2, 
                         boosting_type='goss', 
                         objective='binary',
                         scale_pos_weight = 11.387150050352467,
                         random_state = 924,
                         n_jobs = 2,
                         silent = False,
#                          importance_type (string, optional (default='split')) – 
#                          The type of feature importance to be filled into feature_importances_. 
#                          If ‘split’, result contains numbers of times the feature is used in a model. 
#                          If ‘gain’, result contains total gains of splits which use the feature.
                        )

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 200)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [11]:
init_time = time.time()
params = { 
          'num_leaves': (7, 99), 
          'max_depth': (7, 99),
          'learning_rate': (.001, .1),
#           'n_estimators':(50, 1000),
#           'subsample_for_bin':(50000, 1000000),
#           'min_split_gain': (.01, .1),
          'min_child_weight': (0.0001, 10000),
          'min_child_samples': (7, 99),
#         subsample (float, optional (default=1.)) – Subsample ratio of the training instance.
#         subsample_freq (int, optional (default=0)) – Frequence of subsample, <=0 means no enable.
#         colsample_bytree (float, optional (default=1.)) – Subsample ratio of columns when constructing each tree.
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'max_bin': (55, 511)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 10, n_iter = 10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

|   iter    |  target   | learni... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.724    |  0.002639 |  109.8    |  92.53    |  16.24    |  3.069e+0 |  26.44    |  0.2798   |  0.9827   |
|  2        |  0.7745   |  0.04289  |  457.0    |  20.11    |  74.55    |  3.062e+0 |  77.4     |  0.2327   |  0.3448   |
|  3        |  0.7208   |  0.001806 |  204.2    |  98.95    |  55.32    |  2.583e+0 |  98.65    |  0.07439  |  0.3418   |
|  4        |  0.7762   |  0.08496  |  233.5    |  31.24    |  8.851    |  6.756e+0 |  42.82    |  0.05741  |  0.01844  |
|  5        |  0.7706   |  0.04724  |  490.5    |  97.96    |  8.038    |  5.833e+0 |  18.32    |  0.533    |  0.3083   |
|  6        |  0.7588   |  0.02773  |  383.2    |  92.62    |  37.9     |  6.769e+0 |  87.57    |  0.03484  |  0.2765   |
|  7        |  0.781    

In [12]:
params_list = bo.res
len(params_list)

20

In [13]:
params_list

[{'target': 0.7240041188398739,
  'params': {'learning_rate': 0.0026388572729419127,
   'max_bin': 109.75837740031656,
   'max_depth': 92.52676941089382,
   'min_child_samples': 16.242187970834138,
   'min_child_weight': 3069.139360894056,
   'num_leaves': 26.435768572226436,
   'reg_alpha': 0.2798483684397779,
   'reg_lambda': 0.9826569949568936}},
 {'target': 0.774530936538364,
  'params': {'learning_rate': 0.04288668340447255,
   'max_bin': 457.0334685100756,
   'max_depth': 20.106492356003507,
   'min_child_samples': 74.54642215203191,
   'min_child_weight': 3061.596251895912,
   'num_leaves': 77.40061847332761,
   'reg_alpha': 0.23268076151407524,
   'reg_lambda': 0.3447554271079787}},
 {'target': 0.720849271346437,
  'params': {'learning_rate': 0.0018055663268454142,
   'max_bin': 204.19613397395784,
   'max_depth': 98.94723065531548,
   'min_child_samples': 55.319422234069336,
   'min_child_weight': 2583.384090985445,
   'num_leaves': 98.65036833685474,
   'reg_alpha': 0.0743919

In [15]:
params_list[11]

{'target': 0.7844264336962248,
 'params': {'learning_rate': 0.09388886270968518,
  'max_bin': 508.3052855715264,
  'max_depth': 14.985487906937466,
  'min_child_samples': 10.206446597075596,
  'min_child_weight': 36.720788816817496,
  'num_leaves': 54.98017754883143,
  'reg_alpha': 0.3574409894099684,
  'reg_lambda': 0.3714293857237966}}

In [ ]:
# no need for looping
# params_list = [bo.res[i] for i in range(10)]

In [14]:
# 輸出參數檔案
import json
with open('../../../BDSE12-Group3/datasets/homecdt_ss_output/params_list_BayesOpt_20200213a.txt', 'w', encoding='utf-8') as fout:
    for params in params_list:
        json.dump(params, fout) 
        fout.write("\n")

---

In [ ]:
# 讀取參數檔案
with open('../../../BDSE12-Group3/datasets/homecdt_ss_output/params_list_BayesOpt_20200210a.txt', 'r', encoding='utf-8') as f:
    params_list_read = list(map(json.loads,f))

In [ ]:
params_list_read[8]

In [ ]:
type(params_list_read[8])

In [ ]:
init_time = time.time()
params = {'learning_rate': (.0, .1), 
          'num_leaves': (20, 100), 
          'subsample': (.0, 1.0), 
          'max_depth': (6, 9), 
          'reg_alpha': (.00, 1.0), 
          'reg_lambda': (.00, 1.0), 
          'min_split_gain': (.0, .1),
          'min_child_weight': (20, 70)}
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points = 5, n_iter = 10)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))

---

## XGboost

In [ ]:
def xgb_evaluate(**params):
    warnings.simplefilter('ignore')
    
    params['max_depth'] = int(params['max_depth'])
        
    clf = XGBClassifier(**params, 
                        n_estimators = 2000, 
                        nthread = 5, 
                        objective= 'binary:logistic')

    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]

    folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=1001)
        
    test_pred_proba = np.zeros(train_df.shape[0])
    
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        clf.fit(train_x, train_y, 
                eval_set = [(train_x, train_y), (valid_x, valid_y)], eval_metric = 'auc', 
                verbose = False, early_stopping_rounds = 100)

        test_pred_proba[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]
        
        del train_x, train_y, valid_x, valid_y
        gc.collect()

    return roc_auc_score(train_df['TARGET'], test_pred_proba)

In [ ]:
init_time = time.time()
params = {'learning_rate': (.01, .03), 
          'subsample': (.0, 1.0), 
          'max_depth': (4, 9), 
          'reg_alpha': (.0, 1.0), 
          'reg_lambda': (.0, 1.0), 
          'scale_pos_weight': (.0, 5.0),
          'colsample_bytree': (.0, 1.0)}
bo = BayesianOptimization(xgb_evaluate, params)
bo.maximize(init_points = 5, n_iter = 5)
print("Elapsed time={:5.2f} sec.".format(time.time() - init_time))